In [1]:
# Downloading tinyshakesphere for training
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > tinyshakespeare.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1953k      0 --:--:-- --:--:-- --:--:-- 1952k


# 1. Inspecting the tinyshakespeare text file for training

In [45]:
# Inspecting the text file
with open('tinyshakespeare.txt','r') as file:
    text = file.read()

In [46]:
print(f'There are {len(text)} characters in the dataset')

There are 1115394 characters in the dataset


In [47]:
# Printing the first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [48]:
# Identifying the number of unique characters contained in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Number of unique characters (including white space): {vocab_size}{''.join(chars)}")

Number of unique characters (including white space): 65
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


# 2. Basic mapping between characters to integers

More sophisticated examples include Google's SentencePiece and OpenAI's tiktoken

In [49]:
# Assigning numbers to each characters to encode the characters to integers
ctoi = {char : num for num, char in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
print(encode('Shakespeare in digits'))

# Reversely, decode integers back to characters
itoc = {num : char for num, char in enumerate(chars)}
decode = lambda l : ''.join([itoc[i] for i in l])
print(decode(encode('Shakespeare in digits')))

[31, 46, 39, 49, 43, 57, 54, 43, 39, 56, 43, 1, 47, 52, 1, 42, 47, 45, 47, 58, 57]
Shakespeare in digits


In [50]:
# Tokenizing the total text. Adapted the code to work with Tensorflow instead of pytorch
import tensorflow as tf
data = tf.convert_to_tensor(encode(text))
print(data.shape, data.dtype)
print(data[:100])

(1115394,) <dtype: 'int32'>
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int32)


In [51]:
# Train and validation split sets, with 9:1 ratio
n = int(0.9*len(data))
data_train = data[:n]
data_test = data[n:]
print(f'Length of train data : {len(data_train)}\nLength of test data : {len(data_test)}')

Length of train data : 1003854
Length of test data : 111540


In [52]:
# Starting with block_size implementation
block_size = 8
print(data_train[:block_size + 1])
x = data_train[:block_size]
y = data_train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Input : {context}, Output : {target}')

tf.Tensor([18 47 56 57 58  1 15 47 58], shape=(9,), dtype=int32)
Input : [18], Output : 47
Input : [18 47], Output : 56
Input : [18 47 56], Output : 57
Input : [18 47 56 57], Output : 58
Input : [18 47 56 57 58], Output : 1
Input : [18 47 56 57 58  1], Output : 15
Input : [18 47 56 57 58  1 15], Output : 47
Input : [18 47 56 57 58  1 15 47], Output : 58


In [73]:
## To be worked on : packaging the code with script with variables for later
# Depiction of the chunk(or in here, block)-wise transformation.
# Having varied blocksize allows the algorithm to take into account the context for inference purpose



tf.random.set_seed(1337) # To be sure to have consistent random number
batch_size = 4 # The number of independent sequences to train in parallel
block_size = 8 # The maximum context length for prediction

def get_batch(split):
    data = data_train if split == 'train' else data_test
    # Retrieving batches randomly
    ix = tf.random.uniform(shape = (batch_size,),
                           maxval = len(data) - block_size,
                           dtype = tf.int32)
    # Stacking the list of tensors
    x = tf.stack([data[i:i+block_size] for i in ix])
    y = tf.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)
for batch in range(batch_size):
    for block in range(block_size):
        context = xb[batch, :block+1]
        target = yb[batch, block]
        print(f'When input is {context.numpy().tolist()} the target is {target}')

tf.Tensor(
[[ 1 51 63  1 41 53 39 58]
 [39 42  0 20 47 57  1 52]
 [32 53  1 56 43 60 43 50]
 [54 39 52 63  1 54 47 43]], shape=(4, 8), dtype=int32)
tf.Tensor(
[[51 63  1 41 53 39 58  6]
 [42  0 20 47 57  1 52 39]
 [53  1 56 43 60 43 50  1]
 [39 52 63  1 54 47 43 41]], shape=(4, 8), dtype=int32)
When input is [1] the target is 51
When input is [1, 51] the target is 63
When input is [1, 51, 63] the target is 1
When input is [1, 51, 63, 1] the target is 41
When input is [1, 51, 63, 1, 41] the target is 53
When input is [1, 51, 63, 1, 41, 53] the target is 39
When input is [1, 51, 63, 1, 41, 53, 39] the target is 58
When input is [1, 51, 63, 1, 41, 53, 39, 58] the target is 6
When input is [39] the target is 42
When input is [39, 42] the target is 0
When input is [39, 42, 0] the target is 20
When input is [39, 42, 0, 20] the target is 47
When input is [39, 42, 0, 20, 47] the target is 57
When input is [39, 42, 0, 20, 47, 57] the target is 1
When input is [39, 42, 0, 20, 47, 57, 1] the targ

In [74]:
eval_iters = 200

# A function to average up the loss in multiple batches for both splits
# @tf.function : removing the code, despite slower performance as it causes an error
def estimate_loss():
    output = {}
    model.trainable = False # Setting the model to evaluation phase
    for split in ['train','val']:

        losses = []
        for _ in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model.call(X,Y)
            losses.append(loss)
        output[split] = tf.reduce_mean(losses)
    model.trainable = True # Setting the model back to training phase
    return output

In [75]:
tf.random.set_seed(1337)

class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, vocab_size)

    def call(self, idx, targets=None):
        '''Method for loss calculation, based on idx (input token indices) and
        target (target token indices)
        B : Batch size
        T : Time = sequence length = block size
        C : Channel = number of classes = vocab size
        '''
        logits = self.token_embedding_table(idx)  # Replacing embedding to the indices

        if targets is None:
            loss = None
        else:
            # Reshaping the tensor so that it's compatible with categorical cross entropy
            B, T, C = tf.shape(logits) # Get the shape of logits
            logits = tf.reshape(logits, (B * T, C)) # Flatten logits for comparison
            targets = tf.reshape(targets, (B * T,)) # Flatten targets
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        Text generating method
        '''
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample prediction from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64)

#            idx_next = tf.random.categorical(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, tf.cast(idx_next, tf.int32)], axis=1)  # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model.call(xb, yb)
print(logits.shape)
print(loss.numpy())

print(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))

(32, 65)
4.1713395

saTf-Wz-K-?hNl?Yr:r'LUFLHH:QmLboClI
oYwnqePrE
!zgz'T:,?ZgzxEjHtgpzAQjGjM&vv.;NBdqFlP qxcwcexWhPKs:$&


In [76]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate = 0.001)

In [77]:
batch_size = 32
for step in tf.range(5000):
    if step % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    with tf.GradientTape() as tape:
        logits, loss = model(xb,yb)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print(f'Final Loss: {loss.numpy()}')

Step 0: train loss 4.1737, val loss 4.1742
Step 200: train loss 3.7220, val loss 3.7261
Step 400: train loss 3.4000, val loss 3.4116
Step 600: train loss 3.1756, val loss 3.1860
Step 800: train loss 3.0120, val loss 3.0264
Step 1000: train loss 2.8866, val loss 2.9022
Step 1200: train loss 2.8006, val loss 2.8112
Step 1400: train loss 2.7380, val loss 2.7469
Step 1600: train loss 2.6697, val loss 2.6901
Step 1800: train loss 2.6504, val loss 2.6523
Step 2000: train loss 2.6071, val loss 2.6196
Step 2200: train loss 2.5895, val loss 2.6057
Step 2400: train loss 2.5699, val loss 2.5761
Step 2600: train loss 2.5506, val loss 2.5637
Step 2800: train loss 2.5497, val loss 2.5517
Step 3000: train loss 2.5290, val loss 2.5375
Step 3200: train loss 2.5274, val loss 2.5421
Step 3400: train loss 2.5145, val loss 2.5199
Step 3600: train loss 2.4991, val loss 2.5185
Step 3800: train loss 2.4987, val loss 2.5146
Step 4000: train loss 2.4866, val loss 2.5113
Step 4200: train loss 2.4924, val loss 2.

In [78]:
# Generate a sequence
idx = tf.zeros((1, 1), dtype=tf.int32)
generated_sequence = model.generate(idx, max_new_tokens=1000).numpy()
print(decode(generated_sequence[0].tolist()))


I this issthomfisever ar inkend t jourwhigs w
Ang I puithin l anoft were.
ADILOrd; w aves y mer I I at, hit, re hthainon'nof S:

Vel t fatr oclonc.


Why ayo tsind' oyO:
RUn, t sth;ORAdear t'strince hot,
WBughe!ny, gr'dd bond,
PO:
Whe m'sk be l bth! s wie: nd thimal, ld dero
vKA and b s
Prtholoncct t m Y$CHeaco hfrd h.
D th tomy is peatertur kent ssin; whot w'Thoouedse mythet? h.
BOMy heathad t o chare:

Antowirdo ESI t

QUCKENThecke ce awad tou n amyo gbe be chl:
Hid!pe ty be hequ,
NAn ts RY dowimat e im fout be br'd lit'l:

The inest h ant wimen boruno ir w sisthe thergey h?
LAy, p sid hetrt?
Thee.
NGI hathavesewesterowenthe whiswan l sos th:
puthekind uds, for tht leand t mokeabar t blanoe!eatll'Sy
A:
Wharslasorees amesowngrpamis whais!

mat aker gend a
Foyome aukyolit, fofr'sxpan:
Sel f llmbllie utheay
Be'd thageabl r yo iof w b
SThentor acartondind d; prelog;
I th ad d; t o byrcou, y wor ingay! Lo trindicketrouswind.

Buprdathindextho y halloure.
SCIIVOF$Ducerprhe f t se thir,Yo 